# Drive


In [1]:
from os.path import join
from google.colab import drive
 
ROOT = "/content/drive"
drive.mount(ROOT)
 
PROJ = "My Drive/DetectConcealGun" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)
 
%cd ~/content
%cd drive/My Drive/DetectConcealGun

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
[Errno 2] No such file or directory: '/root/content'
/content
/content/drive/My Drive/DetectConcealGun


In [2]:
cd ep2/app

/content/drive/My Drive/DetectConcealGun/ep2/app


In [3]:
ls

admin_views.py   ConcealGunDetection.ipynb  files/        model.pkl   views.py
admin_views.pyc  data/                      __init__.py   static/     views.pyc
C8ud3_vQcw4.mp4  detect.py                  __init__.pyc  templates/


# New Section

In [4]:
!pip install flask-ngrok

In [20]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import render_template, request, redirect, flash, url_for
from werkzeug.utils import secure_filename
import os
import subprocess
import json
import pandas as pd
import numpy as np
import glob
import cv2
import pickle
from sklearn.preprocessing import StandardScaler
import shutil


app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

app.config["VIDEO_UPLOADS"] = "/content/drive/My Drive/DetectConcealGun/ep2/app/files"
app.config["ALLOWED_VIDEO_EXTENSIONS"] = ["MP4","MOV","AVI"]



def framestoVDO():
  path = 'app/files/images/'
  img_array = []
  for filename in glob.glob(path+'*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img) 
  out = cv2.VideoWriter('app/files/result.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()
  print("it's success, you GREAT!")

def person_box(APIC):
    aray = list()
    for f in glob.glob('app/files/image/*.jpg'):
        aa = (os.path.split(f)[-1])
        aray.append(aa)

    shutil.rmtree('app/files/images')
    os.mkdir('app/files/images')
    path = 'app/files/image/'
    for i in aray : ##
        images = cv2.imread(path+i,cv2.IMREAD_COLOR)

        person = APIC[APIC[0]==i]
        r =  [1.0,2.0,3.0,4.0,5.0]
        #Pose position person /
        for m in r :
            per = person[person[52] == m]
            if len(per) > 0:
                #select body point
                X_min = int(min(min(per[range(4,52,3)].values.tolist())))-50
                X_max = int(max(max(per[range(4,52,3)].values.tolist())))+100
                Y_min = int(min(min(per[range(2,52,3)].values.tolist())))-50
                Y_max = int(max(max(per[range(2,52,3)].values.tolist())))+100
                clas = int(per['Class']) #
                if clas == 1:
                    color = (0,0,255) #RED
                    thickness = 20
                    images = cv2.rectangle(images, (X_max,Y_min), (X_min,Y_max), color, thickness)
                    scor = float("%.4f"% per['Class_1'])*100
                    name = 'Gun'+str(clas)+': '+str(scor)+'%'
                    font = cv2.FONT_HERSHEY_SIMPLEX 
                    fontScale = 2
                    # Line thickness of 2 px 
                    thicknes = 7
                    images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA) 
                else :
                    color = (0, 255, 0) #GREEN
                    thickness = 20
                    images = cv2.rectangle(images,(X_max,Y_min), (X_min,Y_max), color, thickness)
                    scor = float("%.4f"% per['Class_0'])*100
                    name = 'None'+str(clas)+': '+str(scor)+'%'
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 2
                    # Line thickness of 2 px 
                    thicknes = 7
                    images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA)
                cv2.imwrite('app/files/images/'+i, images)
    framestoVDO()


def data_join (file1, file2):
  #load Class
  Class = pd.read_csv(file1) #2041(Class), 2042(Scores)
  #load APIC
  APIC = pd.read_csv(file2,header=None) 
  r =  [1.0,2.0,3.0,4.0,5.0]
  m = range(0,5)
  A = APIC[52]
  B = APIC[52]#*Sores0
  C = APIC[52]#*Sores1
  for n in m :
    di = Class[Class['Unnamed: 0']==n]
    if len(di) > 0:
      clas = int(di['0_Predict'])
      cl = int(di['Unnamed: 0'])
      score_0 = float(di['Class_0'])
      score_1 = float(di['Class_1'])
      if cl == 0:
        A = A.replace(1.0,clas)
        B = B.replace(1.0,score_0)
        C = C.replace(1.0,score_1)
      elif cl == 1:
        A = A.replace(2.0,clas)
        B = B.replace(2.0,score_0)
        C = C.replace(2.0,score_1)
      elif cl == 2:
        A = A.replace(3.0,clas)
        B = B.replace(3.0,score_0)
        C = C.replace(3.0,score_1)
      elif cl == 3:
        A = A.replace(4.0,clas)
        B = B.replace(4.0,score_0)
        C = C.replace(4.0,score_1)
      elif cl == 4:
        A = A.replace(5.0,clas)
        B = B.replace(5.0,score_0)
        C = C.replace(5.0,score_1)

  APIC['Class'] = A
  APIC['Class_0'] = B
  APIC['Class_1'] = C
  #check person
  person = len(Class)
  APICto = APIC[APIC[52] <= person]
  person_box(APICto)

def mode_RF(Data): 
  #reset index
  Data.reset_index(drop=True, inplace=True) 
  X_test = Data.values.tolist()#dataframe to list
  
  #Load model
  filename = "app/model.pkl"
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  
  #prediction
  y_pred= model.predict(X_test)

  # show the inputs and predicted outputs
  P_data=list()
  for i in range(len(X_test)):
    P_out = y_pred[i]
    P_data.append(P_out) 
  # Calling DataFrame constructor on list 
  P_datadf = pd.DataFrame(P_data) 
  # check class
  Data.reset_index(drop=True, inplace=True)#reset index
  Data.reset_index(inplace=True) #set key
  P_datadf.reset_index(inplace=True)#set key
  # keep the predictions for class 
  df = Data.join(P_datadf, on='index', how='left', rsuffix='_Predict') #join left
  df = df.drop(columns=['index_Predict']) #drop don't select

    
  #prop each class
  prob = model.predict_proba(X_test)
  prob_out_df = list()
  for i in range(len(X_test)):
    prob_out = prob[i]
    prob_out_df.append(prob_out) 
  # Calling DataFrame constructor on list 
  prob_df = pd.DataFrame(prob_out_df) 
  prob_df.reset_index(drop=True, inplace=True)#reset index
  prob_df.reset_index(inplace=True) #set key
  prob_df = prob_df.rename(columns={0: "Class_0", 1: "Class_1"}) #ChangName
  df = df.join(prob_df, on='index', how='left', rsuffix='_')
  df = df.drop(columns=['index','index_'])

  #Save .CSV
  df.to_csv('app/files/result_model_prediction.csv') #save result
  data_join ('app/files/result_model_prediction.csv', 'app/files/save_apic.csv')

def data_train():
  file= 'app/files/Fature_to_Test.csv'
  Data = pd.read_csv(file,header=None)
  mode_RF(Data)

def prepare_idx(idkp):
  maxjpg =[]
  for key, value in idkp.items():
    maxjpg.append(max([int(s) for s in key.split('.') if s.isdigit()]))
  maxjpg = max(maxjpg)
  print('max.jpg = ',maxjpg,'.jpg')

  name_column=['XNose','YNose','XLEye','YLEye','XREye','YREye','XLEar','YLEar','XREar','YREar','XLShoulder','YLShoulder','XRShoulder','YRShoulder','XLElbow','YLElbow','XRElbow','YRElbow','XLWrist','YLWrist','XEWrist','YEWrist','XLHip','YLHip','XRHip','YRHip','XLKnee','YLKnee','XRKnee','YRKnee','XLAnkle','YLAnkle','XRAnkle','YRAnkle']
  featvec = list()
  for idxn in range(1,6,1) :
    # create list name pic .jpg ให้วนลูป
    jpglist  = list()
    pic = ['%s.jpg']
    for f in range(1,maxjpg+1,1): 
      link = pic[0] % (f)
      jpglist.append(link)

    frame, keypoints, idx = [], [] , []
    for key, value in idkp.items():
      picx = idkp[(key)]
      for p in picx:
        frame.append(key)
        keypoints.append(p['keypoints'])
        idx.append(p['idx'])
        df = pd.DataFrame() # create empty dataframe 
        df['frame'] = frame
        df['keypoints'] = keypoints
        df['idx'] = idx
        apic = df
    
    jpglistdf = pd.DataFrame(jpglist, columns=['frame']) # Create jpglist dataframe
    pointn = apic[apic['idx'] == (idxn)]
    allFrame = pd.merge(jpglistdf , pointn[['frame', 'keypoints']], on='frame', how='left')
    # Replace NaN with list [-1,-1,-1, .  .  . ,-1] (51 item totol)
    allFrame.loc[allFrame['keypoints'].isnull(),['keypoints']] = allFrame.loc[allFrame['keypoints'].isnull(),'keypoints'].apply(lambda keypoints: [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
    # Keypoint
    kpall = allFrame['keypoints']
    # remove c 
    keypointls = list()
    for frm in kpall:
      frm = [round(i,3) for i in frm]  # convert a list into list with 3 decimal places
      for i in frm:
        if i <= 1 and i >= 0 :
            frm.remove(i)
      keypointls.append(frm)

    # convert list to array
    keypointar = np.asarray(keypointls)

    #ปรับค่า X,Y Nose
    arrayper = keypointar
    X_Zero = arrayper[:,0] #X point Zero
    Y_Zero = arrayper[:,1] #Y point Zero
    l=list()
    for i in range(0, 34):
      if(i%2) == 0:
        A = abs(arrayper[:,i] - X_Zero)
      else:
        A = abs(arrayper[:,i] - Y_Zero)
      l.append(A)

    #Z Score
    
    scaler = StandardScaler()
    scaler.fit(l)
    lstand = scaler.transform(l)
    datalistdf = pd.DataFrame(lstand, name_column)
    df = datalistdf.T

    #กรณี มากกว่า 60 เฟรม จะเริ่มเก็บจากเฟรมสุดท้าย
    (naa,nbb) = lstand.shape
    D1 = list()
    for r in range(0,34):
      for t in range(nbb-60,nbb):
        dat1 = lstand[r,t]
        D1.append(dat1)
    datalistdf1 = pd.DataFrame(D1)
    df1 = datalistdf1.T
    df2 = df.values.tolist()

    #กรณีที่จับได้น้อยกว่า 50%
    ccv0 = 0
    for cv1 in df2:
      for cv0 in cv1:
        if cv0 == 0:
          ccv0 = ccv0 + 1
    if ccv0 < 1020:
      featvec.append(df1)
  featvec2 = pd.concat(featvec)
  np.savetxt('app/files/Fature_to_Test.csv', featvec2,delimiter=',' )
  np.savetxt('app/files/save_apic.csv',apic,fmt=['%s','%s', '%f'],delimiter=',')
  data_train()

def read_json(): #read json file
  with open('app/files/alphapose-results-forvis-tracked.json','r') as myfile:
    data = myfile.read()
    obj = json.loads(data)
    prepare_idx(obj)


def  get_poseflow(video_input):
  shutil.rmtree('files/image')
  os.mkdir("files/image")
  !ffmpeg  -i $video_input -r 30 files/image/%d.jpg    #Extract images

  shutil.rmtree('files/Outimages')
  os.mkdir ("files/Outimages")
  !pip install -q youtube-dl visdom
  os.chdir( "files/AlphaPose" )
  !python3 demo.py --sp --indir ../image/ --outdir ../.
  !pip2 install munkres==1.0.12   #poseflow
  os.chdir('/content/drive/My Drive/DetectConcealGun/ep2/app/files')
  !python2  AlphaPose/PoseFlow/tracker-general.py --imgdir image/ --in_json alphapose-results.json --out_json alphapose-results-forvis-tracked.json --visdir Outimages/
  os.chdir('/content/drive/My Drive/DetectConcealGun/ep2')
  read_json()

def get_length(filename):
  result = subprocess.run(["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  print('Video length : ' ,float(result.stdout))
  if float(result.stdout) < 2:
    print ("Please upload your video less than 2 second")
  elif float(result.stdout) > 3.5 :
    print ("Sorry this video is more than 2 second video")
  else :
    get_poseflow(filename)

def main(videoName):
  os.chdir("/content/drive/My Drive/DetectConcealGun/ep2/app/")
  # videoName = str(input("Enter Video name : "))
  videoName = 'C8ud3_vQcw4.mp4'
  get_length(videoName)



def allowed_video(filename):
    if not "." in filename:
        return False
    ext = filename.rsplit(".", 1)[1]
    if ext.upper() in app.config["ALLOWED_VIDEO_EXTENSIONS"]:
        return True
    else:
        return False

@app.route("/", methods=["GET", "POST"])
def upload_video():
    if request.method == "POST":
        if request.files:
            video = request.files["video"]
            if video.filename == "":
                # flash("No file", "warning")
                return redirect(request.url)
            if allowed_video(video.filename):
                filename = secure_filename(video.filename)
                video.save(os.path.join(app.config["VIDEO_UPLOADS"], filename))
                # flash("Video uploaded", "success")
                main(filename)
                return redirect(request.url)
                # return redirect(url_for("result_detect",filename=filename))

            else:
                # flash("That file extension is not allowed", "danger")
                return redirect(request.url)
    return render_template("public/upload_video.html")




@app.route("/about")
def about():
    return render_template("public/about.html")
  


  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0397b780.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/Jan/2020 05:29:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 05:29:12] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 05:29:12] "GET /static/js/app.js HTTP/1.1" 200 -
127.0.0.1 - - [31/Jan/2020 05:29:12] "GET /favicon.ico HTTP/1.1" 404 -


Video length :  2.0
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --

[2020-01-31 05:33:36,568] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-20-8a2d395c69e1>", line 342, in upload_video
    main(filename)
  File "<ipython-input-20-

# DETECT

In [0]:
import os
import subprocess
import json
import pandas as pd
import numpy as np
import glob
import cv2
import pickle
from sklearn.preprocessing import StandardScaler
import shutil

def framestoVDO():
  path = 'app/files/images/'
  img_array = []
  for filename in glob.glob(path+'*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img) 
  out = cv2.VideoWriter('app/files/result.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()
  print("it's success, you GREAT!")

def person_box(APIC):
    aray = list()
    for f in glob.glob('app/files/image/*.jpg'):
        aa = (os.path.split(f)[-1])
        aray.append(aa)

    shutil.rmtree('app/file/images')
    os.mkdir('app/file/images')
    path = 'app/files/image/'
    for i in aray : ##
        images = cv2.imread(path+i,cv2.IMREAD_COLOR)

        person = APIC[APIC[0]==i]
        r =  [1.0,2.0,3.0,4.0,5.0]
        #Pose position person /
        for m in r :
            per = person[person[52] == m]
            if len(per) > 0:
                #select body point
                X_min = int(min(min(per[range(4,52,3)].values.tolist())))-50
                X_max = int(max(max(per[range(4,52,3)].values.tolist())))+100
                Y_min = int(min(min(per[range(2,52,3)].values.tolist())))-50
                Y_max = int(max(max(per[range(2,52,3)].values.tolist())))+100
                clas = int(per['Class']) #
                if clas == 1:
                    color = (0,0,255) #RED
                    thickness = 20
                    images = cv2.rectangle(images, (X_max,Y_min), (X_min,Y_max), color, thickness)
                    scor = float("%.4f"% per['Class_1'])*100
                    name = 'Gun'+str(clas)+': '+str(scor)+'%'
                    font = cv2.FONT_HERSHEY_SIMPLEX 
                    fontScale = 2
                    # Line thickness of 2 px 
                    thicknes = 7
                    images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA) 
                else :
                    color = (0, 255, 0) #GREEN
                    thickness = 20
                    images = cv2.rectangle(images,(X_max,Y_min), (X_min,Y_max), color, thickness)
                    scor = float("%.4f"% per['Class_0'])*100
                    name = 'None'+str(clas)+': '+str(scor)+'%'
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    fontScale = 2
                    # Line thickness of 2 px 
                    thicknes = 7
                    images = cv2.putText(images, name, (X_max,Y_min-20), font, fontScale, color, thicknes, cv2.LINE_AA)
                cv2.imwrite('app/files/images/'+i, images)
    framestoVDO()


def data_join (file1, file2):
  #load Class
  Class = pd.read_csv(file1) #2041(Class), 2042(Scores)
  #load APIC
  APIC = pd.read_csv(file2,header=None) 
  r =  [1.0,2.0,3.0,4.0,5.0]
  m = range(0,5)
  A = APIC[52]
  B = APIC[52]#*Sores0
  C = APIC[52]#*Sores1
  for n in m :
    di = Class[Class['Unnamed: 0']==n]
    if len(di) > 0:
      clas = int(di['0_Predict'])
      cl = int(di['Unnamed: 0'])
      score_0 = float(di['Class_0'])
      score_1 = float(di['Class_1'])
      if cl == 0:
        A = A.replace(1.0,clas)
        B = B.replace(1.0,score_0)
        C = C.replace(1.0,score_1)
      elif cl == 1:
        A = A.replace(2.0,clas)
        B = B.replace(2.0,score_0)
        C = C.replace(2.0,score_1)
      elif cl == 2:
        A = A.replace(3.0,clas)
        B = B.replace(3.0,score_0)
        C = C.replace(3.0,score_1)
      elif cl == 3:
        A = A.replace(4.0,clas)
        B = B.replace(4.0,score_0)
        C = C.replace(4.0,score_1)
      elif cl == 4:
        A = A.replace(5.0,clas)
        B = B.replace(5.0,score_0)
        C = C.replace(5.0,score_1)

  APIC['Class'] = A
  APIC['Class_0'] = B
  APIC['Class_1'] = C
  #check person
  person = len(Class)
  APICto = APIC[APIC[52] <= person]
  person_box(APICto)

def mode_RF(Data): 
  #reset index
  Data.reset_index(drop=True, inplace=True) 
  X_test = Data.values.tolist()#dataframe to list
  
  #Load model
  filename = "app/model.pkl"
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  
  #prediction
  y_pred= model.predict(X_test)

  # show the inputs and predicted outputs
  P_data=list()
  for i in range(len(X_test)):
    P_out = y_pred[i]
    P_data.append(P_out) 
  # Calling DataFrame constructor on list 
  P_datadf = pd.DataFrame(P_data) 
  # check class
  Data.reset_index(drop=True, inplace=True)#reset index
  Data.reset_index(inplace=True) #set key
  P_datadf.reset_index(inplace=True)#set key
  # keep the predictions for class 
  df = Data.join(P_datadf, on='index', how='left', rsuffix='_Predict') #join left
  df = df.drop(columns=['index_Predict']) #drop don't select

    
  #prop each class
  prob = model.predict_proba(X_test)
  prob_out_df = list()
  for i in range(len(X_test)):
    prob_out = prob[i]
    prob_out_df.append(prob_out) 
  # Calling DataFrame constructor on list 
  prob_df = pd.DataFrame(prob_out_df) 
  prob_df.reset_index(drop=True, inplace=True)#reset index
  prob_df.reset_index(inplace=True) #set key
  prob_df = prob_df.rename(columns={0: "Class_0", 1: "Class_1"}) #ChangName
  df = df.join(prob_df, on='index', how='left', rsuffix='_')
  df = df.drop(columns=['index','index_'])

  #Save .CSV
  df.to_csv('app/files/result_model_prediction.csv') #save result
  data_join ('app/files/result_model_prediction.csv', 'app/files/save_apic.csv')

def data_train():
  file= 'app/files/Fature_to_Test.csv'
  Data = pd.read_csv(file,header=None)
  mode_RF(Data)

def prepare_idx(idkp):
  maxjpg =[]
  for key, value in idkp.items():
    maxjpg.append(max([int(s) for s in key.split('.') if s.isdigit()]))
  maxjpg = max(maxjpg)
  print('max.jpg = ',maxjpg,'.jpg')

  name_column=['XNose','YNose','XLEye','YLEye','XREye','YREye','XLEar','YLEar','XREar','YREar','XLShoulder','YLShoulder','XRShoulder','YRShoulder','XLElbow','YLElbow','XRElbow','YRElbow','XLWrist','YLWrist','XEWrist','YEWrist','XLHip','YLHip','XRHip','YRHip','XLKnee','YLKnee','XRKnee','YRKnee','XLAnkle','YLAnkle','XRAnkle','YRAnkle']
  featvec = list()
  for idxn in range(1,6,1) :
    # create list name pic .jpg ให้วนลูป
    jpglist  = list()
    pic = ['%s.jpg']
    for f in range(1,maxjpg+1,1): 
      link = pic[0] % (f)
      jpglist.append(link)

    frame, keypoints, idx = [], [] , []
    for key, value in idkp.items():
      picx = idkp[(key)]
      for p in picx:
        frame.append(key)
        keypoints.append(p['keypoints'])
        idx.append(p['idx'])
        df = pd.DataFrame() # create empty dataframe 
        df['frame'] = frame
        df['keypoints'] = keypoints
        df['idx'] = idx
        apic = df
    
    jpglistdf = pd.DataFrame(jpglist, columns=['frame']) # Create jpglist dataframe
    pointn = apic[apic['idx'] == (idxn)]
    allFrame = pd.merge(jpglistdf , pointn[['frame', 'keypoints']], on='frame', how='left')
    # Replace NaN with list [-1,-1,-1, .  .  . ,-1] (51 item totol)
    allFrame.loc[allFrame['keypoints'].isnull(),['keypoints']] = allFrame.loc[allFrame['keypoints'].isnull(),'keypoints'].apply(lambda keypoints: [-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])
    # Keypoint
    kpall = allFrame['keypoints']
    # remove c 
    keypointls = list()
    for frm in kpall:
      frm = [round(i,3) for i in frm]  # convert a list into list with 3 decimal places
      for i in frm:
        if i <= 1 and i >= 0 :
            frm.remove(i)
      keypointls.append(frm)

    # convert list to array
    keypointar = np.asarray(keypointls)

    #ปรับค่า X,Y Nose
    arrayper = keypointar
    X_Zero = arrayper[:,0] #X point Zero
    Y_Zero = arrayper[:,1] #Y point Zero
    l=list()
    for i in range(0, 34):
      if(i%2) == 0:
        A = abs(arrayper[:,i] - X_Zero)
      else:
        A = abs(arrayper[:,i] - Y_Zero)
      l.append(A)

    #Z Score
    
    scaler = StandardScaler()
    scaler.fit(l)
    lstand = scaler.transform(l)
    datalistdf = pd.DataFrame(lstand, name_column)
    df = datalistdf.T

    #กรณี มากกว่า 60 เฟรม จะเริ่มเก็บจากเฟรมสุดท้าย
    (naa,nbb) = lstand.shape
    D1 = list()
    for r in range(0,34):
      for t in range(nbb-60,nbb):
        dat1 = lstand[r,t]
        D1.append(dat1)
    datalistdf1 = pd.DataFrame(D1)
    df1 = datalistdf1.T
    df2 = df.values.tolist()

    #กรณีที่จับได้น้อยกว่า 50%
    ccv0 = 0
    for cv1 in df2:
      for cv0 in cv1:
        if cv0 == 0:
          ccv0 = ccv0 + 1
    if ccv0 < 1020:
      featvec.append(df1)
  featvec2 = pd.concat(featvec)
  np.savetxt('app/files/Fature_to_Test.csv', featvec2,delimiter=',' )
  np.savetxt('app/files/save_apic.csv',apic,fmt=['%s','%s', '%f'],delimiter=',')
  data_train()

def read_json(): #read json file
  with open('app/files/alphapose-results-forvis-tracked.json','r') as myfile:
    data = myfile.read()
    obj = json.loads(data)
    prepare_idx(obj)


def  get_poseflow(video_input):
  shutil.rmtree('files/image')
  os.mkdir("files/image")
  !ffmpeg  -i $video_input -r 30 files/image/%d.jpg    #Extract images

  shutil.rmtree('files/Outimages')
  os.mkdir ("files/Outimages")
  !pip install -q youtube-dl visdom
  os.chdir( "files/AlphaPose" )
  !python3 demo.py --sp --indir ../image/ --outdir ../.
  !pip2 install munkres==1.0.12   #poseflow
  os.chdir('/content/gdrive/My Drive/DETECT/TEST PROJECT/app/files')
  !python2  AlphaPose/PoseFlow/tracker-general.py --imgdir image/ --in_json alphapose-results.json --out_json alphapose-results-forvis-tracked.json --visdir Outimages/
  os.chdir('/content/gdrive/My Drive/DETECT/TEST PROJECT')
  read_json()

def get_length(filename):
  result = subprocess.run(["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  print('Video length : ' ,float(result.stdout))
  if float(result.stdout) < 2:
    print ("Please upload your video less than 2 second")
  elif float(result.stdout) > 3.5 :
    print ("Sorry this video is more than 2 second video")
  else :
    get_poseflow(filename)

def main():
  os.chdir("/content/gdrive/My Drive/DETECT/TEST PROJECT/app")
  # videoName = str(input("Enter Video name : "))
  videoName = 'C8ud3_vQcw4.mp4'
  get_length(videoName)


main()



In [12]:
os.getcwd()

'/content/drive/My Drive/DetectConcealGun/ep2/app'

In [0]:
# import os
# from os.path import exists, join, basename, splitext
# import sys
# import matplotlib
# import matplotlib.pylab as plt

# def download_from_google_drive(file_id, file_name):
#   # download a file from the Google Drive link
#   !rm -f ./cookie
#   !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
#   confirm_text = !awk '/download/ {print $NF}' ./cookie
#   confirm_text = confirm_text[0]
#   !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# def dl_pretrain_models(projectname):
#   pretrained_model_path = join(projectname, 'models/sppe/duc_se.pth')
#   if not exists(pretrained_model_path):
#     # download the pretrained model
#     download_from_google_drive('1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW', pretrained_model_path)
#   yolo_pretrained_model_path = join(projectname, 'models/yolo/yolov3-spp.weights')
#   if not exists(yolo_pretrained_model_path):
#     # download the YOLO weights
#     download_from_google_drive('1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC', yolo_pretrained_model_path) 

# def install_alphapose():
#   os.chdir( "app/files" )
#   git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
#   project_name = splitext(basename(git_repo_url))[0]
#   if not exists(project_name):
#     # clone and install dependencies
#     !git clone -q -b pytorch --depth 1 $git_repo_url
#     !cd $project_name && pip install -q -r requirements.txt
#     !pip install -q youtube-dl visdom
#   sys.path.append(project_name)
#   plt.rcParams["axes.grid"] = False
#   dl_pretrain_models(project_name)


# install_alphapose()

     |████████████████████████████████| 484.0MB 37kB/s 
     |████████████████████████████████| 686kB 56.1MB/s 
ERROR: Could not find a version that satisfies the requirement ntpath (from -r requirements.txt (line 9)) (from versions: none)
ERROR: No matching distribution found for ntpath (from -r requirements.txt (line 9))
     |████████████████████████████████| 1.8MB 4.5MB/s 
     |████████████████████████████████| 204kB 35.4MB/s 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1739      0 --:--:-- --:--:-- --:--:--  1739
100  227M    0  227M    0     0  66.0M      0 --:--:--  0:00:03 --:--:-- 75.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1813      0 --:--:-- --:--:-- --:--:--  1813
100  240M  

In [0]:
cd ..

/content/gdrive/My Drive/DetectingConcealedGun
